In [20]:
# Import libraries
suppressMessages({
  library(dplyr)
  library(rjson)
  library(rapsimng)
  library(ggplot2)
  library(future)
  library(future.apply)
  library(parallel)
  library(lhs)
  library(reticulate)
})

source(file.path("..", "R", "init.R"))
init(multicores = 3)

🟢 ApsimX Models folder = /usr/local/bin/Models
🟢 Multicores = 3


# Generate or load sensi folder

In [11]:
sensi_tmp_folder <- generate_sensi_folder(
  folderpath = "../tmp_sensi_1",
  copy_met_data_from = "../met_data/",
  overwrite = TRUE
)

# sensi_tmp_folder <- sensi_folder_load(
#   file.path(CONST_BASE_FOLDER, "tmp_sensi_20250418_212812")
# )

sensi_tmp_folder

🟢 Folder /home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp_sensi_1 was created! 


[1] "/home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp_sensi_1"

# Load problem

In [12]:
problem <- load_problem_from_current_folder()

# problem <- load_problem(sensi_tmp_folder)

problem

🟢 'problem.R' file exists on
/home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/sensitivity folder! Loading... 


$num_vars
[1] 17

$names
 [1] "phen_VegTherTimeResp_X3"   "phen_RepTherTimeResp_X3"  
 [3] "phen_VegPhoMod_X1"         "phen_RepPhoMod_X1"        
 [5] "phen_VegetativeTarget"     "phen_EarlyFloweringTarget"
 [7] "phen_EarlyPodDevTarget"    "phen_FractGrainFill"      
 [9] "phen_MidGrainFill"         "phen_EntGrainFill"        
[11] "phen_Maturing"             "phen_Ripening"            
[13] "leaf_RUE"                  "leaf_AreaLargLeaf"        
[15] "leaf_Phyllochron"          "leaf_ExtinctionCoef_Y1"   
[17] "grain_HarvIndex"          

$bounds
$bounds[[1]]
[1] 21 39

$bounds[[2]]
[1] 21 39

$bounds[[3]]
[1] 10.5 14.5

$bounds[[4]]
[1] 10.5 14.5

$bounds[[5]]
[1] 200 600

$bounds[[6]]
[1] 100 300

$bounds[[7]]
[1]  70 210

$bounds[[8]]
[1] 0.035 0.065

$bounds[[9]]
[1] 0.35 0.65

$bounds[[10]]
[1] 250 750

$bounds[[11]]
[1] 35 65

$bounds[[12]]
[1] 35 65

$bounds[[13]]
[1] 0.8 1.6

$bounds[[14]]
[1] 0.004 0.008

$bounds[[15]]
[1] 35 65

$bounds[[16]]
[1] 0.4 0.8

$bounds[[17]]
[1] 0.35 0.65

# Generate or load samples

In [13]:
# options(repr.plot.width = 14, repr.plot.height = 10)

generate_samples_csv(
  save_csv_to_folder = sensi_tmp_folder,
  problem = problem,
  method = "SOBOL",
  N_SAMPLES = 10L,
  overwrite = FALSE
)

samples_df <- load_samples(
  save_csv_to_folder = sensi_tmp_folder,
  plot = FALSE
)
head(samples_df)

🟢 'samples.csv' file doesn't exist. Generating... 
🟢 File 'samples.csv' was created! 
🟢 Files 'problem.R' and 'problem.rds' were created! 
🟢 Loading samples from
/home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp_sensi_1/samples.csv 


,id,phen_VegTherTimeResp_X3,phen_RepTherTimeResp_X3,phen_VegPhoMod_X1,phen_RepPhoMod_X1,phen_VegetativeTarget,phen_EarlyFloweringTarget,phen_EarlyPodDevTarget,phen_FractGrainFill,phen_MidGrainFill,phen_EntGrainFill,phen_Maturing,phen_Ripening,leaf_RUE,leaf_AreaLargLeaf,leaf_Phyllochron,leaf_ExtinctionCoef_Y1,grain_HarvIndex
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,32.56420,37.6001,12.14400,13.31499,406.4629,258.8332,75.63381,0.05085097,0.4162313,421.908,49.73264,46.14215,1.324622,0.006448676,44.0806,0.4793985,0.5386574
2,2,36.58213,37.6001,12.14400,13.31499,406.4629,258.8332,75.63381,0.05085097,0.4162313,421.908,49.73264,46.14215,1.324622,0.006448676,44.0806,0.4793985,0.5386574
3,3,32.56420,27.1621,12.14400,13.31499,406.4629,258.8332,75.63381,0.05085097,0.4162313,421.908,49.73264,46.14215,1.324622,0.006448676,44.0806,0.4793985,0.5386574
4,4,32.56420,37.6001,10.75758,13.31499,406.4629,258.8332,75.63381,0.05085097,0.4162313,421.908,49.73264,46.14215,1.324622,0.006448676,44.0806,0.4793985,0.5386574
5,5,32.56420,37.6001,12.14400,11.83145,406.4629,258.8332,75.63381,0.05085097,0.4162313,421.908,49.73264,46.14215,1.324622,0.006448676,44.0806,0.4793985,0.5386574
6,6,32.56420,37.6001,12.14400,13.31499,420.7431,258.8332,75.63381,0.05085097,0.4162313,421.908,49.73264,46.14215,1.324622,0.006448676,44.0806,0.4793985,0.5386574


# Generate all samples simulations

In [16]:
generate_apsimxs(
  sensi_folder = sensi_tmp_folder,
  sensit_base_sim_filepath = file.path("../base_simulations/Soy-BR-fields-MG6REAL.apsimx"),
  runs_only_some_n = 20L,
  parallel = FALSE,
  dry_run = FALSE
)

🟢 Loading 'samples.csv' from folder
/home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp_sensi_1 
🟢 Generating 20 samples...
🟢 Not using parallel 
  |======================================================================| 100%
📌 Summary:
📌   Folder = /home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp_sensi_1/sims_and_met
📌   Number of csvs: 0
📌   Number of dbs: 0
📌   Number of apsimxs: 20


# Run all simulations in parallel

In [19]:
run_apsimxs(
  sensi_folder = sensi_tmp_folder,
  runs_only_some_n = 2L,
  # ids_to_run = c(1, 20, 10, 11, 16, 15),
  parallel = FALSE,
  dry_run = FALSE
)

🟢 Running 2 apsimx simulations...
🟢 Not using parallel 
  |======================================================================| 100%
📌 Summary:
📌   Folder = /home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp_sensi_1/sims_and_met
📌   Number of csvs: 0
📌   Number of dbs: 2
📌   Number of apsimxs: 20


# Summarize db files

In [26]:
source(file.path("..", "R", "init.R"))
init(multicores = 3)

summarized_df <- summarize_harvest_dbs(
  sensi_folder = sensi_tmp_folder,
  # ids_to_summarize = c(1, 2),
  runs_only_some_n = 5L,
  parallel = FALSE,
  # number_of_fields = 32L,
  overwrite = TRUE,
  dry_run = FALSE
)
head(summarized_df)

🟢 ApsimX Models folder = /usr/local/bin/Models
🟢 Multicores = 3


🟡 WARNING: 'summarized.csv' file already exists on
/home/luanabeckerdaluz/git/luanabeckerdaluz/apsimx-sensitivity/tmp_sensi_1 folder! Summarize
process will run because overwrite parameter was set to TRUE.
🟡 WARNING: WARNING: runs_only_some_n parameter [5] must be lower or equal than db files count [2].
Updating its value to 2
🟢 Not using parallel 
  |======================================================================| 100%
🟢 File 'summarized.csv' was overwriten! 


,id,field,yield,biomass,maturity
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,1,CR16,5498.759,12774.04,158
2,1,CR17,5762.167,13018.13,164
3,1,CR18,6699.862,15190.34,168
4,1,FW14a,6199.044,14371.11,161
5,1,FW14b,6181.857,14071.67,157
6,1,FW14c,5472.073,12220.11,151


# Compute SALib

In [69]:
# salib_for_one_field_and_param(
#     field = "CA14",
#     df = read.csv(file.path(sensi_tmp_folder, "summarized.csv")),
#     param = "yield",
#     number_of_simulations = 360,
#     problem = problem,
#     salib_sobol = TRUE,
#     fix_NAs_with_mean = TRUE,
#     dry_run = FALSE
#   ) |>
#   head()

source(file.path("..", "R", "init.R"))
init(multicores = 3)

df_salib <- compute_salib_for_all_params_and_fields(
  sensi_tmp_folder = sensi_tmp_folder,
  salib_sobol = TRUE,
  fix_NAs_with_mean = TRUE,
  # fields = c("CA14", "CA15"),
  # params = c("yield", "biomass"),
  dry_run = FALSE,
  overwrite = TRUE,
  parallel = TRUE
)
head(df_salib)

🟢 Base folder = /home/luanabeckerdaluz/git/luanabeckerdaluz/apsimtestes
🟢 Met folder = /home/luanabeckerdaluz/git/luanabeckerdaluz/apsimtestes/met_data
🟢 Base simulations folder = /home/luanabeckerdaluz/git/luanabeckerdaluz/apsimtestes/base_simulations
🟢 ApsimX Models folder = /usr/local/bin/Models
🟢 Multicores = 3


🟡 WARNING: 'salib.csv' file already exists on
/home/luanabeckerdaluz/git/luanabeckerdaluz/apsimtestes/tmp_sensi_20250214_224536 folder! However,
it will be overwritten because 'overwrite' parameter was set as TRUE!
🟢 Running in parallel with 3 cores 
🟢 Loading 'samples.csv' from
/home/luanabeckerdaluz/git/luanabeckerdaluz/apsimtestes/tmp_sensi_20250214_224536/samples.csv 
🟢 Loading 'summarized.csv' from
/home/luanabeckerdaluz/git/luanabeckerdaluz/apsimtestes/tmp_sensi_20250214_224536/summarized.csv 
🟢 Loading 'problem.R' from
/home/luanabeckerdaluz/git/luanabeckerdaluz/apsimtestes/tmp_sensi_20250214_224536/problem.R 
🟡 WARNING: field=FW14a param=yield -> number_of_simulations [180] is greater than array length
[3]. Filling with 177 NAs...
🟡 WARNING: field=CR18 param=yield -> number_of_simulations [180] is greater than array length [3].
Filling with 177 NAs...
🟡 WARNING: field=CR16 param=yield -> number_of_simulations [180] is greater than array length [3].
Filling with 177 NAs...
🟡 WAR

,field,param,names,S1,S1_conf,ST,ST_conf
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,FW14a,yield,phen_VegTherTimeResp_X3,2.571939e-14,2.730006e-14,6.6757913,7.0860728
2,FW14a,yield,phen_RepTherTimeResp_X3,5.758767e-15,6.112690e-15,0.3346881,0.3552574
3,FW14a,yield,phen_VegPhoMod_X1,1.049272e-14,1.113758e-14,1.1111111,1.1793979
4,FW14a,yield,phen_RepPhoMod_X1,1.049272e-14,1.113758e-14,1.1111111,1.1793979
5,FW14a,yield,phen_VegetativeTarget,1.049272e-14,1.113758e-14,1.1111111,1.1793979
6,FW14a,yield,phen_EarlyFloweringTarget,1.049272e-14,1.113758e-14,1.1111111,1.1793979
